In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import numpy as np
import requests

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

In [4]:
API_KEY = '902635857ca292a7bc2c9a8ac5aa773b'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'


In [5]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    
    return {
        'city': data.get('name'),
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'wind_gust_speed': data['wind']['speed']
    }

In [6]:
def read_historical_data(filename):
    df = pd.read_csv(filename)
    df = df.dropna()
    df = df.drop_duplicates()
    return df

In [7]:
def prepare_data(data):
    le = LabelEncoder()
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

    X = data.drop(columns='RainTomorrow')
    y = data['RainTomorrow']

    return X, y, le

In [8]:
def train_rain_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    model = GradientBoostingClassifier(n_estimators=100, random_state = 42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print("Mean Squared Error for Rain model")
    print(mean_squared_error(y_test, y_pred))

    return model

In [9]:
def prepare_regression_data(data, feature):
    X, y = [],[]

    for i in range(len(data)-1):
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i+1])

    X = np.array(X).reshape(-1,1)
    y = np.array(y)

    return X, y

In [10]:
def train_regression_model(X, y):
    model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    model.fit(X,y)
    return model

In [11]:
def predict_future(model, current_value):
    predictions = [current_value]

    for i in range(5):
        next_value = model.predict(np.array([[predictions[-1]]]))
        predictions.append(next_value[0])

    return predictions[1:]

In [16]:
def weather_view():

    city = "wardha"
    current_weather = get_current_weather(city)
    historical_data = read_historical_data("/kaggle/input/weather/weather.csv")

    X, y, le = prepare_data(historical_data)

    rain_model = train_rain_model(X,y)

    wind_deg = current_weather['wind_gust_dir'] % 360
    campass_points = [
        ("N",0,11.25), ("NNE",11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 1911.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)    
    ]
    campass_direction = next(point for point, start, end in campass_points if start <= wind_deg < end)
    
    campass_direction_encoded = le.transform([campass_direction])[0] if campass_direction in le.classes_ else -1

    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'], 
        'WindGustDir': campass_direction_encoded,
        'WindGustSpeed': current_weather['wind_gust_speed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp']
    }

    current_df = pd.DataFrame([current_data])

    rain_prediction = rain_model.predict(current_df)[0]

    X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(X_temp, y_temp)

    hum_model = train_regression_model(X_hum, y_hum)


    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    timezone = pytz.timezone('Asia/Kolkata')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours = 1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

    future_times = [(next_hour + timedelta(hours = i)).strftime("%H:00") for i in range(5)]

    print(f"City: {city},{current_weather['country']}")
    print(f"Current Temp: {current_weather['current_temp']}")
    print(f"Feels like: {current_weather['feels_like']}")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Prediction:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")
        
    print("\nFuture Humidity Prediction:")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

weather_view()

Mean Squared Error for Rain model
0.1506849315068493
City: wardha,IN
Current Temp: 25
Feels like: 25
Minimum Temperature: 25°C
Maximum Temperature: 25°C
Humidity: 44%
Weather Prediction: broken clouds
Rain Prediction: No

Future Temperature Prediction:
12:00: 26.2°C
13:00: 27.3°C
14:00: 26.2°C
15:00: 27.3°C
16:00: 26.2°C

Future Humidity Prediction:
12:00: 45.1%
13:00: 45.8%
14:00: 47.9%
15:00: 52.2%
16:00: 45.6%
